In [1]:
# Establish connections to the input db and variables spreadsheets

# Import modules
import os, re, math
import arcpy
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from dotenv import load_dotenv

# Lot the .env variables
load_dotenv()

# Set the data paths
inputs_path = os.getenv('J111_INPUTS')
outputs_path = os.getenv('J111_OUTPUTS')
spatial_path = os.getenv('J111_SPATIAL')
env_path = os.getenv('J111_ENV')

# Set path to the input variables excel workbook
vars_xlsx = os.path.join(inputs_path, 'Redland_GAM_Input_Vars.xlsx')

# Get full path of Inputs GDB
input_gdb_path = os.path.join(spatial_path, 'redland_gam_current_dev.gdb')

# Set arcpy workspace
arcpy.env.workspace = os.path.join(env_path, 'J111_redland_gam.gdb')

# Import the Council Property features into dataframe
land = pd.DataFrame.spatial.from_featureclass(os.path.join(input_gdb_path, 'INPUT_FC_CouncilProperty')).drop(['OBJECTID', 'SHAPE'], axis=1)

# Keep only the necessary fields
land = land.loc[:, ('landno', 'zone', 'prop_use')]

# Inspect dataframe
land.head()

,landno,zone,prop_use
0,998033,LDR,RES001
1,998037,LDR,RES001
2,998036,LDR,RES001
3,998038,LDR,RES001
4,998039,LDR,RES001


In [2]:
''' 
----------------------------------
Merge in gfa data for each land parcel
----------------------------------
'''

# Import the Building Footprints data
bldg = pd.DataFrame.spatial.from_featureclass(os.path.join(input_gdb_path, 'INPUT_FC_CouncilProperty_BldgFootprints')).drop(['OBJECTID', 'SHAPE'], axis=1)

# Calculate GFA for each footprint
bldg['gfa_est'] = bldg['bldg_ftprnt_area'] * bldg['bldg_floors_est']

# Groupby landno and sum gfa for each land parcel
bldg = bldg.loc[:, ('landno', 'gfa_est')].groupby(by='landno').sum().reset_index()

# Merge gfa to land table
land_bldg = pd.merge(land, bldg, on='landno', how='left')

# Set no gfa to be 0
land_bldg['gfa_est'] = land_bldg['gfa_est'].fillna(0)

# Inspect
land_bldg.head()

,landno,zone,prop_use,gfa_est
0,998033,LDR,RES001,403.57
1,998037,LDR,RES001,50.71
2,998036,LDR,RES001,246.32
3,998038,LDR,RES001,449.07
4,998039,LDR,RES001,35.54


In [3]:
''' 
----------------------------------
Join the land data to the DCDB reference
----------------------------------
'''
# Import the Council Property to DCDB Base lots relational list
parcels_to_land = pd.DataFrame.spatial.from_table(os.path.join(input_gdb_path, 'INPUT_TB_DCDB_CouncilPropertyRel')).drop(['OBJECTID'], axis=1)

# Select only the required column
parcels_land = parcels_to_land.loc[:, ('segpar', 'relationships', 'landnos')].rename({'relationships' : 'rels'}, axis=1)

# Split the land numbers string into list
parcels_land.landnos = parcels_land.landnos.apply(lambda v: [int(x) for x in v.split(';')])

# Explode on the land numbers column (this will create duplicate parcel segpar records)
parcels_land = parcels_land.explode('landnos', ignore_index=True).rename({'landnos' : 'landno'}, axis=1)

# Merge in land data
parcels_land = pd.merge(parcels_land.loc[:, ('landno', 'segpar')], land_bldg, on='landno', how='left')

# Inspect
parcels_land.head()

,landno,segpar,zone,prop_use,gfa_est
0,146617,28663092,LDR,RES001,326.72
1,996564,28460061,LDR,RES014,0.02
2,145846,32043083,CR,RES001,227.08
3,144907,20484023,CN,VAC003,0.00
4,112272,20689095,LDR,RES001,306.54


In [4]:
''' 
----------------------------------
Set missing propuse on select parcels
----------------------------------
'''

# Import property use / development groups table from input
missing_propuse = pd.read_excel(vars_xlsx, 'MissingPropertyUse')

# Set the missing propuse on input table
for index, row in missing_propuse.iterrows():
    parcels_land.loc[parcels_land['landno'] == row.landno, 'prop_use'] = row.prop_use

# Inspect
missing_propuse.head()

,landno,prop_use
0,101613,RES005
1,103040,RES005
2,995718,RES005
3,991532,RES005
4,106900,VAC003


In [5]:
''' 
----------------------------------
Incorporate development measures
----------------------------------
'''

# Import property use / development groups table from input
devuse = pd.read_excel(vars_xlsx, 'PropertyUse_to_DevGroup')

# Join details to parcels table
parcels_devuse = pd.merge(parcels_land, devuse, on='prop_use', how='left')

# Define non residential columns
dev_cols = {
    'Detached Dwelling' : 'det_dwl',
    'Multiple Dwelling' : 'att_dwl',
    'Accommodation - Long-Term' : 'accom_long',
    'Accommodation - Short-Term' : 'accom_short',
    'Emergency Services' : 'emer_serv',
    'Places of Assembly' : 'pl_of_ass',
    'Childcare' : 'childcare',
    'Education' : 'education',
    'Care Accommodation' : 'care_accom',
    'Hospital' : 'hospital',
    'Health Services' : 'health_serv',
    'Heavy and Special Industrial' : 'heavy_ind',
    'Light and General Industrial' : 'light_ind',
    'Office' : 'office',
    'Other' : 'other',
    'Rural' : 'rural',
    'Bulky Goods' : 'bulk_goods',
    'Food and Drink' : 'food_drink',
    'Indoor Entertainment and Recreation' : 'indoor_ent',
    'Indoor Recreation (Court Areas)' : 'indoor_rec',
    'Retail and Services' : 'retail'
}

# Add devgroup columns
for k, v in dev_cols.items():
    parcels_devuse[v] = 0.0

# Inspect
parcels_devuse.head()

C:\Users\Daniel Smith\AppData\Local\ESRI\conda\envs\arcgis-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,landno,segpar,zone,prop_use,gfa_est,prop_type,tag,res_devgroup,nonres_devgroup,nonres_devgroup_alt,...,heavy_ind,light_ind,office,other,rural,bulk_goods,food_drink,indoor_ent,indoor_rec,retail
0,146617,28663092,LDR,RES001,326.72,Dwelling house (Approved),Residential,Detached Dwelling,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,996564,28460061,LDR,RES014,0.02,Body Corporate (Residential),Common,NaN,Not Applicable,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,145846,32043083,CR,RES001,227.08,Dwelling house (Approved),Residential,Detached Dwelling,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,144907,20484023,CN,VAC003,0.00,Vacant land,No Development,NaN,Not Applicable,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,112272,20689095,LDR,RES001,306.54,Dwelling house (Approved),Residential,Detached Dwelling,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
''' 
----------------------------------
Run non-residential calculations per land parcel
----------------------------------
'''

# Select only those non-residential uses
parcels_devuse_nonres = parcels_devuse.loc[
    (parcels_devuse['tag'] == 'Non-Residential') | \
    (parcels_devuse['tag'] == 'Allocate even') | \
    (parcels_devuse['tag'] == 'Allocate by zone') | \
    (parcels_devuse['tag'] == 'Split') \
]

# Drop redundant columns
parcels_devuse_nonres = parcels_devuse_nonres.drop(['prop_type', 'res_devgroup', 'measure', 'unit_count', 'unit_size_sqm'], axis=1)

# Allocate gfa by tag and devtype
for index, row in parcels_devuse_nonres.iterrows():

    gfa = row.gfa_est * 0.7 # 30% reduction in GFA estimate to account for non-contributory areas (walls & circulation)
    
    if row.tag == 'Allocate even':

        parcels_devuse_nonres.at[index, dev_cols[row.nonres_devgroup]] = gfa * 0.5
        parcels_devuse_nonres.at[index, dev_cols[row.nonres_devgroup_alt]] = gfa * 0.5

    if row.tag == 'Allocate by zone':

        if len([zone for zone in str(row.zone).split(',') if zone in ['LI','MI','WMI']]):

            parcels_devuse_nonres.at[index, dev_cols[row.nonres_devgroup]] = gfa

        else:

            parcels_devuse_nonres.at[index, dev_cols[row.nonres_devgroup_alt]] = gfa

    else:

        parcels_devuse_nonres.at[index, dev_cols[row.nonres_devgroup]] = gfa

# Drop rows
parcels_devuse_nonres = parcels_devuse_nonres.drop(['gfa_est', 'zone', 'tag', 'nonres_devgroup', 'nonres_devgroup_alt'], axis=1)
        
# Inspect
parcels_devuse_nonres.head()

,landno,segpar,prop_use,det_dwl,att_dwl,accom_long,accom_short,emer_serv,pl_of_ass,childcare,...,heavy_ind,light_ind,office,other,rural,bulk_goods,food_drink,indoor_ent,indoor_rec,retail
56,302373,63023029,INF013,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,885.514,0.0,0.0,0.0,0.0,0.0,0.0
213,345405,30549362,COM045,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
229,337030,20567564,COM045,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
255,200623,28634045,COM010,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
288,115261,30513029,COM009,0.0,0.0,0.0,0.0,0.0,0.0,467.18,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
''' 
----------------------------------
Run residential calculations per land parcel
----------------------------------
'''

# Select only those non-residential uses
parcels_devuse_res = parcels_devuse.loc[
    (parcels_devuse['tag'] == 'Residential') | \
    (parcels_devuse['tag'] == 'Split') \
]

# Drop redundant columns
parcels_devuse_res = parcels_devuse_res.drop(['prop_type', 'nonres_devgroup', 'nonres_devgroup_alt'], axis=1)

# Calc by measure and count
for index, row in parcels_devuse_res.iterrows():

    gfa = row.gfa_est * 0.7 # 30% reduction in GFA estimate to account for non-contributory areas (walls & circulation)
    
    if row.measure == 'Unit':

        parcels_devuse_res.at[index, dev_cols[row.res_devgroup]] = math.ceil(gfa / row.unit_size_sqm)

    elif row.measure == 'Count':

        parcels_devuse_res.at[index, dev_cols[row.res_devgroup]] = row.unit_count

    else: # Measure = GFA

        parcels_devuse_res.at[index, dev_cols[row.res_devgroup]] = gfa

# Drop rows
parcels_devuse_res = parcels_devuse_res.drop(['tag', 'gfa_est', 'zone', 'res_devgroup', 'measure', 'unit_count', 'unit_size_sqm'], axis=1)
        
# Inspect
parcels_devuse_res.head()

,landno,segpar,prop_use,det_dwl,att_dwl,accom_long,accom_short,emer_serv,pl_of_ass,childcare,...,heavy_ind,light_ind,office,other,rural,bulk_goods,food_drink,indoor_ent,indoor_rec,retail
0,146617,28663092,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,145846,32043083,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,112272,20689095,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,330770,63124016,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,143890,20889026,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
''' 
----------------------------------
Combine all land parcel calculations into single dataframe
----------------------------------
'''

# Select non development uses
parcels_devuse_nodev = parcels_devuse.loc[
    (parcels_devuse['tag'] == 'Common') | \
    (parcels_devuse['tag'] == 'No Development') \
]

# Drop redundant columns
parcels_devuse_nodev = parcels_devuse_nodev.drop(['tag', 'gfa_est', 'zone', 'prop_type', 'nonres_devgroup', 'nonres_devgroup_alt', 'res_devgroup', 'measure', 'unit_count', 'unit_size_sqm'], axis=1)

# Merge residential, non-reseidential and no development tables
parcels_dev = pd.concat([parcels_devuse_res, parcels_devuse_nonres, parcels_devuse_nodev])

# Inspect
parcels_dev.head()

,landno,segpar,prop_use,det_dwl,att_dwl,accom_long,accom_short,emer_serv,pl_of_ass,childcare,...,heavy_ind,light_ind,office,other,rural,bulk_goods,food_drink,indoor_ent,indoor_rec,retail
0,146617,28663092,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,145846,32043083,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,112272,20689095,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,330770,63124016,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,143890,20889026,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
''' 
----------------------------------
Split parcels into aggregates and otherwise
----------------------------------
'''

# Import the DCDB aggregates table
aggregates = pd.DataFrame.spatial.from_table(os.path.join(input_gdb_path, 'GEN_TB_DCDB_Aggregates')).drop(['OBJECTID'], axis=1)

# Join the aggid onto the land parcels
parcels_dev_agg = pd.merge(parcels_dev, aggregates, on='segpar', how='left')

# split the aggregate and non-aggregate parcels
parcels_dev_noagg = parcels_dev_agg.loc[parcels_dev_agg['aggid'].isnull()].drop(['aggid'], axis=1).reset_index(drop=True)
parcels_dev_agg = parcels_dev_agg.loc[parcels_dev_agg['aggid'].notnull()].reset_index(drop=True)

# Inspect
parcels_dev_agg.head()

,landno,segpar,prop_use,det_dwl,att_dwl,accom_long,accom_short,emer_serv,pl_of_ass,childcare,...,light_ind,office,other,rural,bulk_goods,food_drink,indoor_ent,indoor_rec,retail,aggid
0,145036,32041024,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AGG001759
1,117547,20514039,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AGG000240
2,113842,30591118,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AGG002829
3,366825,20647074,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AGG000054
4,360235,20516039,RES001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AGG000243


In [10]:
''' 
----------------------------------
Handle development calculations for aggregate properties
----------------------------------
'''

# Import the DCDB aggregate details table
agg_details = pd.DataFrame.spatial.from_table(os.path.join(input_gdb_path, 'GEN_TB_DCDB_AggregateDetails')).drop(['OBJECTID'], axis=1)

# Get all holding aggregates
holdings = agg_details.query("aggtype == 'holding'")

# Get all holding land numbers as set
holding_landnos = {*holdings['landno'].to_list()}

# Select all land parcels that are part of a property holding
parcels_dev_agg_holding = parcels_dev_agg.query("landno in @holding_landnos").drop(['landno', 'segpar', 'prop_use'], axis=1)

# For each holding aggregate select the maximum value across all development measures
agg_holdings = parcels_dev_agg_holding.groupby('aggid').agg('max').reset_index()

# Select all other non-holding parcels
parcels_dev_agg_noholding = parcels_dev_agg.query("landno not in @holding_landnos").drop(['landno', 'segpar', 'prop_use'], axis=1)

# For each other aggregate sum all development measures
agg_other = parcels_dev_agg_noholding.groupby('aggid').agg('sum').reset_index()

# For non-holding aggregates all dwellings are measured as attached stock
agg_other['att_dwl'] = agg_other['det_dwl'] + agg_other['att_dwl']
agg_other['det_dwl'] = 0.0

# Concatenate all aggregates into single table
agg_all = pd.concat([agg_holdings, agg_other]).sort_values(by='aggid')

# For each aggregate compile the property use details
agg_propuse = parcels_dev_agg.loc[:, ('aggid', 'prop_use')].groupby('aggid')['prop_use'].apply(set).reset_index()

# Convert the prop_use column from a set to a string
agg_propuse['prop_use'] = agg_propuse['prop_use'].str.join(',')

# Merge the property use details back onto the processed aggregates
dev_agg = pd.merge(agg_propuse, agg_all, on='aggid', how='left')

# Inspect
dev_agg.head()

,aggid,prop_use,det_dwl,att_dwl,accom_long,accom_short,emer_serv,pl_of_ass,childcare,education,...,heavy_ind,light_ind,office,other,rural,bulk_goods,food_drink,indoor_ent,indoor_rec,retail
0,AGG000001,"INF021,RES001",0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AGG000002,"INF020,IND004",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3404.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AGG000003,"INF021,RES001",0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AGG000004,"INF021,RES001",0.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AGG000005,"INF020,IND004",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,803.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
''' 
----------------------------------
Join the development calculations onto the property base
----------------------------------
'''

# Import the Property Base features
propbase = pd.DataFrame.spatial.from_featureclass(os.path.join(input_gdb_path, 'GEN_FC_PropertyBase')).drop(['OBJECTID'], axis=1)

# Split the propbase by aggregate
propbase_agg = propbase.query("aggid == aggid").loc[:, ('pbno', 'aggid', 'sa2_name')]
propbase_noagg = propbase.query("aggid != aggid").loc[:, ('pbno', 'segpar', 'sa2_name')]

# Join in the development metrics
propbase_agg = pd.merge(propbase_agg, dev_agg, on='aggid', how='left').dropna()
propbase_noagg = pd.merge(propbase_noagg, parcels_dev_noagg.drop(['landno'], axis=1), on='segpar', how='left').dropna()

# Combine aggregate and non aggregate tables
propbase_dev = pd.concat([propbase_agg, propbase_noagg]).drop(['aggid', 'segpar'], axis=1).dropna()

# Merge duplicate rows taking the maximum value in all cases
propbase_dev = propbase_dev.groupby('pbno').agg('max').reset_index()

# Inspect
propbase_dev.head()

,pbno,sa2_name,prop_use,det_dwl,att_dwl,accom_long,accom_short,emer_serv,pl_of_ass,childcare,...,heavy_ind,light_ind,office,other,rural,bulk_goods,food_drink,indoor_ent,indoor_rec,retail
0,8000000,Cleveland,"INF021,RES001",0.0,9.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8000001,Cleveland,"INF020,IND004",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3404.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8000002,Redland Islands,"INF021,RES001",0.0,18.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8000003,Cleveland,"INF021,RES001",0.0,146.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8000004,Capalaba,"INF020,IND004",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,803.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
''' 
----------------------------------
Get the planned capacity summary by SA2
----------------------------------
'''
# Group development statistics by SA2
sa2_dev = propbase_dev.drop(['pbno', 'prop_use'], axis=1).groupby('sa2_name').agg('sum')

# Inspect
sa2_dev.head(20)

,det_dwl,att_dwl,accom_long,accom_short,emer_serv,pl_of_ass,childcare,education,care_accom,hospital,...,heavy_ind,light_ind,office,other,rural,bulk_goods,food_drink,indoor_ent,indoor_rec,retail
sa2_name,,,,,,,,,,,,,,,,,,,,,
Alexandra Hills,5720.0,474.0,123.0,8504.405,0.000,1345.813,1652.238,86797.893,5572.273,0.000,...,0.000,0.000,261.2190,14800.282,0.000,1057.455,0.000,0.0,0.0,28355.782
Birkdale,4837.0,552.0,227.0,7027.601,436.611,3221.925,2854.320,21877.471,17375.589,0.000,...,0.000,17.563,5080.6070,3002.167,0.000,12147.142,0.000,0.0,0.0,25261.460
Capalaba,5478.0,1628.0,0.0,13174.721,1747.130,9881.669,3427.767,77346.990,6823.712,0.000,...,0.000,331908.017,96153.7045,14534.387,0.000,24545.276,4491.676,0.0,0.0,308002.219
Cleveland,4260.0,2761.0,178.0,6146.546,1036.966,26280.674,2323.482,51231.145,16057.090,69947.941,...,0.000,167748.777,73226.2195,20336.554,0.000,2137.548,2132.403,0.0,0.0,103304.817
Ormiston,1841.0,708.0,50.0,2158.177,0.000,9847.159,2156.406,46883.032,0.000,0.000,...,0.000,15657.264,18792.3190,671.811,0.000,19771.703,901.859,0.0,0.0,15088.017
Redland Bay,6084.0,390.0,55.0,4371.899,1782.417,4052.300,2765.098,13572.699,2884.714,0.000,...,117.964,24589.579,7927.6400,6610.037,74028.283,38753.120,0.000,0.0,0.0,9914.261
Redland Islands,6632.0,722.0,3.0,9157.995,3092.124,16241.897,1408.400,16678.984,2303.343,0.000,...,0.000,12032.090,6832.9800,15791.426,0.000,0.000,2320.850,0.0,0.0,19538.764
Sheldon - Mount Cotton,2683.0,42.0,0.0,1064.812,0.000,1566.684,1638.581,41873.041,0.000,0.000,...,14585.424,24763.893,0.0000,4088.735,84338.485,850.514,0.000,0.0,0.0,2128.392
Thorneside,992.0,597.0,0.0,10524.864,0.000,690.333,709.205,0.000,0.000,0.000,...,0.000,13249.257,0.0000,765.191,0.000,0.000,0.000,0.0,0.0,2699.368


In [19]:
''' 
----------------------------------
Output results
----------------------------------
'''
# Join development metrics back onto the probase spatial dataframe
propbase = pd.merge(propbase.loc[:, ('pbno', 'SHAPE')], propbase_dev, on='pbno', how='left').dropna()

# Output probase back into geodatabase
propbase.spatial.to_featureclass(os.path.join(input_gdb_path, 'OUT_FC_PropertyBase_CurrentDev'), overwrite=True)

# Output SA2 metrics to csv
sa2_dev.to_csv(os.path.join(outputs_path, 'OUT_Redland_SA2_CurrentDevelopment.csv'))